Imports

In [1]:
import pandas as pd
import pymongo
import xlsxwriter

from secret import DB_STRING, DATABASE_NAME, COLLECTION_NAME
from global_variables import *

Function name: create_df
Purpose: Turns a mongodb collection into a pandas DataFrame
Input:
    documents: a pymongo cursor containg all the documents to be added to the dataframe
    column_index: The names of the keys in each document, to become columns in the pandas dataframe
Output:
    df: a pandas Dataframe with the given columns and data

In [2]:
def create_df(documents, column_index):
    
    df = pd.DataFrame(columns = column_index)
    for doc in documents:
        df.loc[len(df)] = doc
    return df

Function name: create_spreadsheet
Purpose: Uses Xlsxwriter to turn a pandas DataFrame into an excel file
Input:
    df: the pandas DataFrame to turn into an excel file
    file_name: The name of the excel file to be created
    sheet_name: The name of the excel sheet to write to
Output:
    writer: The xlsx Writer object that is formatting the excel file

In [3]:
def create_spreadsheet(df, file_name, sheet_name):
    writer = pd.ExcelWriter(file_name, engine = 'xlsxwriter')
    df.to_excel(writer, sheet_name = sheet_name, index = False)
    return writer

Function name: create_formats
Purpose: adds and returns formats needed for the excel file
Inputs:
    writer: an xlsxwriter object that is writing to the excel file
    background_color: the background color for each column, defaults to white
    font_color: the font color for each column, defaults to black
Output:
    formats: A dictionary containg key-value pairs with the name of each format and the corresponding format variable

In [4]:
def create_formats(writer, background_color = 'ffffff', font_color = '000000'):
    formats = {}

    string_format = writer.book.add_format(
        {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        }
    )
    formats['string format'] = string_format
    
    integer_format = writer.book.add_format(
        {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
        }
    )
    formats['integer format'] = integer_format
    return formats

Function name: create_format_dictionary
Purpose: Makes a dictionary containing key-value pairs mapping each column in the excel file to its index name and format
Input:
    formats: a dictionary containg the formats to be used
    columns_index: The names of the columns in the excel spreadsheet
Output:
    format_dictionary: the dictionary with columns and their formats

In [5]:
def create_column_dictionary(formats, columns_index):
    format_dictionary = {
        'A': [columns_index[0], formats['string format']],
        'B': [columns_index[1], formats['string format']],
        'C': [columns_index[2], formats['string format']],
        'D': [columns_index[3], formats['integer format']],
        'E': [columns_index[4], formats['string format']]
    }
    return format_dictionary

Function name: apply_formats
Purpose: Applies the proper format to each column of the excel file
Inputs:
    writer: the xlsx writer object that is writing to the excel file
    format_dictionary: a dictionary with the columns of the excel file and their corresponding name and formats
    sheet_name: the name of the excel sheet to write to
Output:
    Void

In [6]:
def apply_formats(writer, column_dictionary, sheet_name):
    for column, formats in column_dictionary.items():
        writer.sheets[sheet_name].set_column(f'{column}:{column}', 30, formats[1])
        writer.sheets[sheet_name].write(f'{column}1', formats[0], formats[1])

Function name: scholarships_to_excel
Purpose: Turns a collection with scholarship information into and excel document
Inputs:
    db_uri: the uri for a port where mongod service is listening
    database_name: the name of the database containing the collection with scholarship data
    collection_name: the name of the collection with scholarship data
    columns: the name of the columns of the excel file
    excel_file_name: the name of the excel file to write to
    excel_sheet_name: the name of the excel sheet to write to

In [9]:
def scholarships_to_excel(db_uri, database_name, collection_name, columns, excel_file_name, excel_sheet_name):
    collection = get_collection(db_uri, database_name, collection_name)
    column_names = collection.find().next().keys()
    documents = collection.find()
    
    df = create_df(documents, columns)
    writer = create_spreadsheet(df, excel_file_name, excel_sheet_name)
    formats = create_formats(writer)
    column_dic = create_column_dictionary(formats, columns)
    apply_formats(writer, column_dic, excel_sheet_name)
    writer.close()

In [8]:
scholarships_to_excel(DB_STRING, DATABASE_NAME, COLLECTION_NAME, COLUMNS, EXCEL_FILE_NAME, EXCEL_SHEET_NAME)